In [405]:
pip install gradio


In [406]:
pip install kmodes

In [407]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [408]:
import gradio as gr
import numpy as np
import pandas as pd
import pickle

from kmodes.kmodes import KModes
from sklearn.metrics.pairwise import cosine_similarity

In [409]:
#import all the files (Male cat, female cat, onehot male and female)
males = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/Roomilarity_dataCAT-Male.csv")
females = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/Roomilarity_dataCAT-Female.csv")
priority_female = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/Roomilarity_dataCAT-PriorityFemale.csv")
priority_male = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/Roomilarity_dataCAT-PriorityMale.csv")

# print(males.shape)
# print(priority_male.shape)

priority_male


,Year Group_Class of 2024,Year Group_Class of 2025,Year Group_Class of 2026,Year Group_Class of 2027,How would you describe your personality?_I am extroverted,How would you describe your personality?_I am introverted,How would you describe your personality?_Somewhere in between,What time do you typically go to bed on weeknights?_10PM or earlier,What time do you typically go to bed on weeknights?_After Midnight (12AM),What time do you typically go to bed on weeknights?_Between 10PM and Midnight (12AM),...,"Do you enjoy having friends over in your room?_Yup, but not always",How would your friends describe you in one word?_Chill,How would your friends describe you in one word?_Excellent,How would your friends describe you in one word?_Friendly,How would your friends describe you in one word?_Fun,"How would your friends describe you in one word?_I see more than just one ,I have about 20% of everything there",How would your friends describe you in one word?_Organized,How would your friends describe you in one word?_Reserved,How would your friends describe you in one word?_Responsible,cluster
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,10.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0,4.0,...,10.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,12
4,0.0,0.0,0.0,6.0,0.0,0.0,9.0,0.0,10.0,0.0,...,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,0.0,0.0,7.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1
211,0.0,0.0,0.0,7.0,0.0,5.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,2
212,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
213,0.0,0.0,8.0,0.0,0.0,0.0,6.0,0.0,7.0,0.0,...,7.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3


In [410]:
with open('/content/drive/MyDrive/Colab Notebooks/Final Project/F_kmodes_model.pkl', 'rb') as file:
    F_model = pickle.load(file)

with open('/content/drive/MyDrive/Colab Notebooks/Final Project/M_kmodes_model.pkl', 'rb') as file:
    M_model = pickle.load(file)

In [411]:
def run_model(info_array,similarity,priority_female,priority_male,males,females):
  data = np.array(info_array[1:])
  data = data.reshape( 1,-1)


  if data[0,0] =='Male':
    group = M_model.predict(data)
    print(data)
    # print(M_model.cluster_centroids_)

    origional_cat_options = [4,3,3,2,3,4,8]
    choices=["Class of 2024", "Class of 2025", "Class of 2026", "Class of 2027", "I am extroverted", "I am introverted", "Somewhere in between",
            "10PM or earlier","After Midnight (12AM)","Between 10PM and Midnight (12AM)","Away from the room",
             "In the room","Going for events on/off-campus", "Hanging out with friends", "Relaxing/Sleeping",
              'Nope!', "Not really, but I don't mind",'Yup, all the time', 'Yup, but not always',"Chill", "Excellent",
             "Friendly", "Fun", "I see more than just one ,I have about 20% of everything there", "Organized", "Reserved", "Responsible"]

  else:
    group = F_model.predict(data)
    print(data)
    # print(F_model.cluster_centroids_)

    origional_cat_options = [4,3,3,2,3,4,10]
    choices=["Class of 2024", "Class of 2025", "Class of 2026", "Class of 2027", "I am extroverted", "I am introverted", "Somewhere in between",
            "10PM or earlier","After Midnight (12AM)","Between 10PM and Midnight (12AM)","Away from the room",
             "In the room","Going for events on/off-campus", "Hanging out with friends", "Relaxing/Sleeping",
              'Nope!', "Not really, but I don't mind",'Yup, all the time', 'Yup, but not always',"All the above mentioned", "Calm","Chill",
             "Friendly", "Fun", "No idea", "Organized","Responsible", "Unsociable", "fun, responsible and organized"]

  new_similarity=[]
  start = 0
  n_index = 0

  one_hot = []

  for i in origional_cat_options:
    new_similarity[start:start+i]=[similarity[n_index]]*i

    for j in range(i):
      if data[0,n_index+1]==choices[start+j]:
        one_hot.append(1)
      else:
        one_hot.append(0)

    n_index += 1
    start += i


  priority = np.array(one_hot)*np.array(new_similarity)
  priority= np.append(priority,group)

  data= np.append(data,group)
  data = data.reshape(1,-1)


  print(data)


  if data[0,0] =='Male':
    df = pd.DataFrame([priority],columns=priority_male.columns)
    priority_male = pd.concat([df,priority_male],ignore_index=True)


    data=np.append(males.shape[0]+1,data)
    data=data.reshape(1,-1)
    df = pd.DataFrame(data,columns=males.columns)
    males = pd.concat([df,males],ignore_index=True)

    males.to_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/Roomilarity_dataCAT-Male.csv",index=False)
    priority_male.to_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/Roomilarity_dataCAT-PriorityMale.csv",index=False)

    wanted_one_hot_data = priority_male[np.array(priority_male['cluster'])==group]

    similarity_matrix = cosine_similarity(wanted_one_hot_data.drop("cluster",axis=1))
    # np.fill_diagonal(similarity_matrix, 0)
    compatibility =[]


    for i in range(similarity_matrix[0].shape[0]-1):
      compatibility.append(similarity_matrix[0][i]*100)
      # if i != 0:
      #   print(f"student {0} is {similarity_matrix[0][i]*100:.2f} % compatible with student {i}")

    wanted_data = males[np.array(males['cluster'])==group].copy()
    wanted_data["Compatibility (%)"] = np.array(compatibility)


    wanted_data.sort_values(by='Compatibility (%)',ascending=False,inplace=True)
    top_5 = wanted_data.head(5)

  else:
    df = pd.DataFrame([priority],columns=priority_female.columns)
    priority_female = pd.concat([df,priority_female],ignore_index=True)

    data=np.append(females.shape[0]+1,data)
    data=data.reshape(1,-1)

    df = pd.DataFrame(data,columns=females.columns)
    females = pd.concat([df,females],ignore_index=True)

    females.to_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/Roomilarity_dataCAT-Female.csv",index=False)
    priority_female.to_csv("/content/drive/MyDrive/Colab Notebooks/Final Project/Roomilarity_dataCAT-PriorityFemale.csv",index=False)

    wanted_one_hot_data = priority_female[np.array(priority_female['cluster'])==group]

    similarity_matrix = cosine_similarity(wanted_one_hot_data.drop("cluster",axis=1))
    # np.fill_diagonal(similarity_matrix, 0)
    compatibility =[]


    for i in range(similarity_matrix[0].shape[0]-1):
      compatibility.append(similarity_matrix[0][i]*100)
      # if i != 0:
      #   print(f"student {0} is {similarity_matrix[0][i]*100:.2f} % compatible with student {i}")


    wanted_data = females[np.array(females['cluster'])==group].copy()
    wanted_data["Compatibility (%)"] = np.array(compatibility)

    wanted_data.sort_values(by='Compatibility (%)',ascending=True,inplace=True)
    top_5 = wanted_data.head(5)

  return top_5

  # print(priority)


# personal_info = np.array([["Dally","Female","C2027","I am extroverted","Between 10PM and Midnight (12AM)","In the room","Relaxing/Sleeping",'Yup, but not always',"Friendly"]])
# priority_info = np.array([[4,7,3,2,7,1,8]])

# # print(priority_info.shape)

# # personal_info = personal_info.reshape(-1,1)


# run_model(personal_info,priority_info,priority_female,priority_male,males,females)


In [412]:
# Definiton of functions
def select_option(choice):
    return f"You selected: {choice}"

def go_to_main_screen():
    # Hide welcome screen, show main screen, hide options screen
    return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)

def go_to_options_screen():
    # Hide main screen, hide welcome screen, show options screen
    return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

def store_and_process_responses(name, dropdownGender, dropdownClass, dropdownDescribe, dropdownBedtime,
                    dropdownStudyLocation, dropdownFun,dropdownFroom,
                    dropdownFriends, compatibility_slider_1,
                    compatibility_slider_2, compatibility_slider_3,
                    compatibility_slider_4, compatibility_slider_5,
                    compatibility_slider_6, compatibility_slider_7):

    personal_info = np.array([name, dropdownGender, dropdownClass, dropdownDescribe,
                              dropdownBedtime, dropdownStudyLocation,
                              dropdownFun,dropdownFroom, dropdownFriends])

    priority_info = np.array([compatibility_slider_1, compatibility_slider_2,
                              compatibility_slider_3, compatibility_slider_4,
                              compatibility_slider_5, compatibility_slider_6,
                              compatibility_slider_7])

    result = run_model(personal_info,priority_info,priority_female,priority_male,males,females)

    # response_array = np.array(responses)
    # return response_array
    return result

In [413]:
with gr.Blocks() as demo:
    # Section 1: Welcome screen
    with gr.Column() as welcome_screen:
        gr.Markdown("""<h1 style="
            font-family: 'Impact', sans-serif;
            font-size: 42px;
            color: #3E5725;
            text-shadow: 2px 2px 8px rgba(0, 0, 0, 0.2);
            letter-spacing: 1px;
            margin-bottom: 20px;
        ">
            Welcome to Roomlarity!
        </h1>
        <p style="font-family: 'Lato', serif; font-size: 20px;"> Whether you're searching for someone who vibes with your lifestyle, or your study habits. <br>Roomlarity helps you match smarter, faster, and better. </p>""")

        continue_button = gr.Button("Continue")

    # Section 2: Main screen
    with gr.Column(visible=False) as main_screen:
        name = gr.Textbox(label="Please enter your full name?", placeholder="Enter your name here...")

        dropdownClass = gr.Dropdown(
            choices=["Class of 2027", "Class of 2026", "Class of 2025", "Class of 2024"],
            label="Pick your below class",
            interactive=True,
        )

        dropdownGender = gr.Dropdown(
            choices = ["Male", "Female"],
            label = "Pick your Gender",
            interactive =True,
        )

        dropdownDescribe = gr.Dropdown(
            choices=["I am extroverted", "I am introverted", "Somewhere in between"],
            label="How would you describe your personality?",
            interactive=True,
        )

        dropdownBedtime = gr.Dropdown(choices=[
            "10PM or earlier",
            "After Midnight (12AM)",
            "Between 10PM and Midnight (12AM)"],
            label="What time do you typically go to bed on weeknights?",
            interactive=True,
        )

        dropdownStudyLocation = gr.Dropdown(choices=["Away from the room", "In the room"],
            label="Where do you prefer to do homework/study?",
            interactive=True,
        )

        dropdownFun = gr.Dropdown(
            choices=["Going for events on/off-campus", "Hanging out with friends", "Relaxing/Sleeping"],
            label="What do you enjoy doing for fun?",
            interactive=True,
        )

        dropdownFroom = gr.Dropdown(
            choices=['Nope!', "Not really, but I don't mind",
                     'Yup, all the time', 'Yup, but not always'],
            label="Do you enjoy having friends over in your room?",
            interactive=True,
        )

        dropdownFriends = gr.Dropdown(
            label="How would your friends describe you in one word?",
            choices=[
                "Chill", "Excellent", "Friendly", "Fun",
                "I see more than just one ,I have about 20% of everything there",
                "Organized", "Reserved", "Responsible"],
            interactive=True
        )

        # Ask for importance of compatibility with a slider (scale 1-10)
        compatibility_slider_1 = gr.Slider(
            label="How important is it that you are paired with someone of the same year group?",
            minimum=1, maximum=10, step=1, interactive=True
        )

        compatibility_slider_2 = gr.Slider(
            label="How important is it that you are paired with someone of the same personality?",
            minimum=1, maximum=10, step=1, interactive=True
        )

        compatibility_slider_3 = gr.Slider(
            label="How important is it that you are paired with someone of the same bedtimes?",
            minimum=1, maximum=10, step=1, interactive=True
        )

        compatibility_slider_4 = gr.Slider(
            label="How important is it that you are paired with someone of the same studyLocation?",
            minimum=1, maximum=10, step=1, interactive=True
        )

        compatibility_slider_5 = gr.Slider(
            label="How important is it that your roommate enjoy doing the same things for fun?",
            minimum=1, maximum=10, step=1, interactive=True
        )

        compatibility_slider_6 = gr.Slider(
            label="How important is it that you are paired with someone of the same preference for inviting friends over to your room?",
            minimum=1, maximum=10, step=1, interactive=True
        )

        compatibility_slider_7 = gr.Slider(
            label="How important is it that you are paired with someone who's friends describe them as yours describe you?",
            minimum=1, maximum=10, step=1, interactive=True
        )

        # Store the responses of all the sliders
        process_button = gr.Button("Process")

        # Textbox to display the selected choices
        select_output = gr.Dataframe(label="Best Choices")


    # Link dropdown to select_option function
    dropdownClass.change(
        fn=select_option,
        inputs=dropdownClass,
        outputs=select_output
    )

    dropdownDescribe.change(
        fn=select_option,
        inputs=dropdownDescribe,
        outputs=select_output
    )

    dropdownBedtime.change(
        fn=select_option,
        inputs=dropdownBedtime,
        outputs=select_output
    )

    dropdownStudyLocation.change(
        fn=select_option,
        inputs=dropdownStudyLocation,
        outputs=select_output
    )

    dropdownFun.change(
        fn=select_option,
        inputs=dropdownFun,
        outputs=select_output
    )

    dropdownFroom.change(
        fn=select_option,
        inputs=dropdownFroom,
        outputs=select_output
    )

    dropdownFriends.change(
        fn=select_option,
        inputs=dropdownFriends,
        outputs=select_output
    )

    dropdownGender.change(
        fn = select_option,
        inputs=dropdownGender,
        outputs=select_output
    )

    # When "Continue" is clicked, hide welcome and show main screen
    continue_button.click(
        fn=go_to_main_screen,
        inputs=[],
        outputs=[welcome_screen, main_screen]
    )

    # Store the responses of the compatibility questions
    process_button.click(
        fn=store_and_process_responses,
        inputs=[name, dropdownGender, dropdownClass, dropdownDescribe, dropdownBedtime,
                dropdownStudyLocation, dropdownFun,dropdownFroom,
                dropdownFriends, compatibility_slider_1, compatibility_slider_2,
                compatibility_slider_3, compatibility_slider_4,
                compatibility_slider_5, compatibility_slider_6,
                compatibility_slider_7],

        outputs=select_output,

        show_progress = True
    )


In [414]:
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cbb4bc5b6b8a553119.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1855: UserWarning: A function (go_to_main_screen) returned too many output values (needed: 2, returned: 3). Ignoring extra values.
    Output components:
        [column, column]
    Output values returned:
        [{'__type__': 'update', 'visible': False}, {'__type__': 'update', 'visible': True}, {'__type__': 'update', 'visible': False}]
  warnings.warn(


[['Male' 'Class of 2027' 'I am extroverted' '10PM or earlier'
  'Away from the room' 'Going for events on/off-campus' 'Nope!' 'Chill']]
[['Male' 'Class of 2027' 'I am extroverted' '10PM or earlier'
  'Away from the room' 'Going for events on/off-campus' 'Nope!' 'Chill'
  '0']]
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> https://cbb4bc5b6b8a553119.gradio.live
